#LLAMAR A LA TABLA DE BRONZE

In [0]:
# Llamar a la tablas de bronze
df_song_silver = spark.table("proyecto_spotify.bronze_songs")
df_artists_silver = spark.table("proyecto_spotify.bronze_artists")

## 1) ENTENDER LA TABLA

In [0]:
# SONG
df_song_silver.printSchema()

# ARTISTS
df_artists_silver.printSchema()

root
 |-- song_id: string (nullable = true)
 |-- song_name: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- popularity: integer (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- spotify_followers: integer (nullable = true)
 |-- youtube_followers: integer (nullable = true)
 |-- active_since: integer (nullable = true)



## 2) ELIMINAR COLUMNAS BASURAS

In [0]:
# Asignarle en una variable df_artistas y eliminar desde data a la columna genre ya que no vamos a utilizar
df_artistas = df_artists_silver.drop("genre")

## 3) ESTANDARIZAR NOMBRES DE COLUMNAS 
- sin espacios
- sin mayusculas
- sin tilde

In [0]:
# songs
df_musica = (
    df_song_silver
    .withColumnRenamed("song_id", "id_cancion")
    .withColumnRenamed("song_name", "nombre_cancion")
    .withColumnRenamed("artist_id", "id_artista")
    .withColumnRenamed("genre", "genero")
    .withColumnRenamed("release_year", "anio_lanzamiento")
    .withColumnRenamed("popularity", "popularidad")
)


# artists
df_artistas = (
    df_artistas
    .withColumnRenamed("active_since", "activo_desde")
    .withColumnRenamed("artist_id", "id_artista")
    .withColumnRenamed("artist_name", "nombre_artista")
    .withColumnRenamed("country", "pais")
    .withColumnRenamed("spotify_followers", "seguidores_spotify")
    .withColumnRenamed("youtube_followers", "seguidores_youtube")
)


## 3.1) ESTANDARIZAR NOMBRES DE VALORES DE COLUMNAS

### TABLA ARTIST, TODAS LAS COLUMNAS DE SUS VALORES

In [0]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType

df_artistas.groupBy("activo_desde").count().orderBy("activo_desde").display() # para ver los años que hay sin editado

df_artistas.withColumn( # remplazar los nombres de los paises
    "activo_desde",
    when(
        (col("activo_desde").rlike("^[0-9]{4}$")) &
        (col("activo_desde").cast(IntegerType()) >= 1950) &
        (col("activo_desde").cast(IntegerType()) <= 2026),
        col("activo_desde").cast(IntegerType())
    )
    .otherwise(None)
)
df_artistas.groupBy("activo_desde").count().orderBy("activo_desde").display()

###########################################################################################################################################################

df_artistas.groupBy("pais").count().orderBy("pais").display() # para ver los paises que hay sin editado

# Tienes que igualar la variable al resultado de la transformación
df_artistas = df_artistas.withColumn(
    "pais",
    when(col("pais") == "Argentina", "Argentina")
    .when(col("pais") == "Barbados", "UNKNOWN")
    .when(col("pais") == "Canada", "Canada")
    .when(col("pais") == "Chile", "Chile")
    .when(col("pais") == "Colombia", "Colombia")
    .when(col("pais") == "Dominican Rep", "Republica Dominicana")
    .when(col("pais") == "France", "Francia")
    .when(col("pais") == "Gustavo", "UNKNOWN")
    .when(col("pais") == "Ireland", "Irlanda")
    .when(col("pais") == "Mexico", "Mexico")
    .when(col("pais") == "Netherlands", "Paises Bajos")
    .when(col("pais") == "Norway", "Noruega")
    .when(col("pais") == "Panama", "Panama")
    .when(col("pais") == "Puerto Rico", "Puerto Rico")
    .when(col("pais") == "South Korea", "Corea del Sur")
    .when(col("pais") == "Spain", "Espana")
    .when(col("pais") == "Sweden", "Suecia")
    .when(col("pais") == "UK", "Reino Unido")
    .when(col("pais") == "USA", "Estados Unidos")
    .when(col("pais") == "USA/Latam", "Estados Unidos")
    .when(col("pais") == "USA/Mexico", "Mexico")
    .when(col("pais") == "USA/Puerto Rico", "Puerto Rico")
    .otherwise(col("pais"))
)

# Ahora sí verás los cambios
df_artistas.groupBy("pais").count().display()

###########################################################################################################################################################

# ESTANDARIZAR NOMBRE ARTISTA
df_artistas.groupBy("nombre_artista").count().orderBy("nombre_artista").display()
# aca esta bien echo todo

###########################################################################################################################################################

from pyspark.sql.types import LongType

df_artistas.groupBy("seguidores_spotify").count().orderBy("seguidores_spotify").display()

df_artistas.withColumn(
    "seguidores_spotify",
     when(col("seguidores_spotify").rlike("^[0-9]+$"),
         col("seguidores_spotify").cast(LongType()))
    .otherwise(None)
)
df_artistas.groupBy("seguidores_spotify").count().orderBy("seguidores_spotify").display()

###########################################################################################################################################################

df_artistas.groupBy("seguidores_youtube").count().orderBy("seguidores_youtube").display()
# aca esta bien echo todo


activo_desde,count
null,1
1960,1
1963,1
1965,1
1968,1
1969,1
1970,1
1973,1
1976,2
1979,1


activo_desde,count
null,1
1960,1
1963,1
1965,1
1968,1
1969,1
1970,1
1973,1
1976,2
1979,1


pais,count
Argentina,16
Barbados,1
Canada,4
Chile,2
Colombia,13
Dominican Rep,2
France,2
Ireland,1
Mexico,15
Netherlands,1


pais,count
Estados Unidos,34
Reino Unido,14
Canada,4
Puerto Rico,26
UNKNOWN,1
Corea del Sur,2
Colombia,13
Suecia,1
Francia,2
Irlanda,1


nombre_artista,count
Adele,1
Alan Walker,1
Anuel AA,1
Arctic Monkeys,1
Arcángel,1
Ariana Grande,1
Avicii,1
BLACKPINK,1
BTS,1
Bad Bunny,1


seguidores_spotify,count
null,1
2700000,1
2800000,2
2900000,1
3000000,2
3100000,1
3200000,2
3300000,1
3400000,2
3500000,2


seguidores_spotify,count
null,1
2700000,1
2800000,2
2900000,1
3000000,2
3100000,1
3200000,2
3300000,1
3400000,2
3500000,2


seguidores_youtube,count
null,1
1200000,3
1800000,1
2100000,1
2400000,1
2600000,1
2800000,2
3100000,2
3300000,1
3600000,1


In [0]:
# para eliminar una fila q esta mal 
df_artistas = df_artistas.filter(col("artist_id") != "artist_id")
df_artistas.display()

id_artista,nombre_artista,pais,seguidores_spotify,seguidores_youtube,activo_desde
AR001,Taylor Swift,Estados Unidos,150000000,43000000,2006
AR002,Ed Sheeran,Reino Unido,124000000,37000000,2011
AR003,Billie Eilish,Estados Unidos,122000000,44000000,2015
AR004,The Weeknd,Canada,117000000,30000000,2010
AR005,Ariana Grande,Estados Unidos,109000000,46000000,2008
AR006,Bad Bunny,Puerto Rico,106000000,10000000,2015
AR007,Drake,Canada,106000000,27000000,2006
AR008,Eminem,Estados Unidos,106000000,36000000,1999
AR009,Justin Bieber,Canada,86500000,67000000,2009
AR010,Rihanna,UNKNOWN,70900000,23000000,2005


### TABLA SONGS - TODAS LAS COLUMNAS DE SUS VALORES

In [0]:
# ENTANDARIZAR EL SONGS
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType

df_musica.groupBy("nombre_cancion").count().orderBy("nombre_cancion").display()
# aca esta bien echo todo

###########################################################################################################################################################

df_musica.groupBy("genero").count().orderBy("genero").display()

df_musica = df_musica.withColumn(
    "genero",
    when(col("genero")   == "Alternative", "Alternativa")
    .when(col("genero")  == "Argentina", "Desconocido")
    .when(col("genero")  == "Electronic", "Electronica")
    .when(col("genero")  == "Hip Hop", "Hip Hop")
    .when(col("genero")  == "K-POP", "K-POP")
    .when(col("genero")  == "Metal", "Metal")
    .when(col("genero")  == "Pop", "Pop")
    .when(col("genero")  == "Pop Latino", "Pop Latino")
    .when(col("genero")  == "Pop Rock", "Pop Rock")
    .when(col("genero")  == "Punk Rock", "Punk Rock")
    .when(col("genero")  == "R&B", "R&B")
    .when(col("genero")  == "Reggaeton", "Regueton")
    .when(col("genero")  == "Rock", "Rock")
    .when(col("genero")  == "Rock Latino", "Rock Latino")
    .when(col("genero")  == "Trap", "Trap")
    .otherwise(col("genero"))
)
df_musica.groupBy("genero").count().orderBy("genero").display()

###########################################################################################################################################################

df_musica.groupBy("anio_lanzamiento").count().orderBy("anio_lanzamiento").display()
# nada q cambiar

nombre_cancion,count
24K Magic,1
7 Rings,1
Adan y Eva,1
Animals,1
Another One Bites The Dust,1
Anti-Hero,1
Antidote,1
Arabella,1
Around The World,1
As It Was,1


genero,count
Alternative,6
Electronic,13
Hip Hop,17
K-Pop,6
Metal,3
Pop,46
Pop Latino,2
Pop Rock,5
Punk Rock,1
R&B,3


genero,count
Alternativa,6
Electronica,13
Hip Hop,17
K-Pop,6
Metal,3
Pop,46
Pop Latino,2
Pop Rock,5
Punk Rock,1
R&B,3


anio_lanzamiento,count
1965,1
1968,1
1970,1
1975,2
1977,1
1979,1
1980,1
1986,2
1987,4
1990,2


In [0]:
df_musica.display()

id_cancion,nombre_cancion,id_artista,genero,anio_lanzamiento,popularidad
S001,Shake It Off,AR001,Pop,2014,95
S002,Shape of You,AR002,Pop,2017,98
S003,Bad Guy,AR003,Pop,2019,92
S004,Blinding Lights,AR004,R&B,2019,99
S005,Thank U Next,AR005,Pop,2018,91
S006,Titi Me Pregunto,AR006,Regueton,2022,96
S007,Gods Plan,AR007,Hip Hop,2018,94
S008,Lose Yourself,AR008,Hip Hop,2002,93
S009,Sorry,AR009,Pop,2015,90
S010,Umbrella,AR010,Pop,2007,89


## 4) CORREGIR TIPOS DE DATOS DE COLUMNAS

In [0]:
# 1. Transformación de Canciones
df_musica.withColumns({
    "id_cancion": col("id_cancion").cast("string"),
    "nombre_cancion": col("nombre_cancion").cast("string"),
    "id_artista": col("id_artista").cast("string"),
    "genero": col("genero").cast("string"),
    "año_lanzamiento": col("año_lanzamiento").cast("integer"), # Solo el año como número
    "popularidad": col("popularidad").cast("integer")
})

df_musica.printSchema()

root
 |-- id_cancion: string (nullable = true)
 |-- nombre_cancion: string (nullable = true)
 |-- id_artista: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- anio_lanzamiento: integer (nullable = true)
 |-- popularidad: integer (nullable = true)



In [0]:
# 2. Transformación de Artistas
df_artistas.withColumns({
    "id_artista": col("id_artista").cast("string"),
    "nombre_artista": col("nombre_artista").cast("string"),
    "pais": col("pais").cast("string"),
    "seguidores_spotify": col("seguidores_spotify").cast("integer"),
    "seguidores_youtube": col("seguidores_youtube").cast("integer"),
    "activo_desde": col("activo_desde").cast("integer") # Solo el año como número
})

df_artistas.printSchema()

root
 |-- id_artista: string (nullable = true)
 |-- nombre_artista: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- seguidores_spotify: integer (nullable = true)
 |-- seguidores_youtube: integer (nullable = true)
 |-- activo_desde: integer (nullable = true)



## 5) MANEJO DE NULOS

### PARA VER CUANTOS NULOS TENEMOS POR COLUMNA

In [0]:
from pyspark.sql.functions import col

# Recorremos todas las columnas del DataFrame
for columna in df_musica.columns:
    
    # Contamos cuántas filas tienen NULL en esa columna
    cantidad_nulos = df_musica.filter(col(columna).isNull()).count()
    
    # Mostramos el nombre de la columna y cuántos nulos tiene
    print(columna, cantidad_nulos)

id_cancion 0
nombre_cancion 0
id_artista 0
genero 0
anio_lanzamiento 0
popularidad 0


In [0]:
from pyspark.sql.functions import col

for columna in df_artistas.columns:
    cantidad_nulos = df_artistas.filter(col(columna).isNull()).count()
    print(columna, cantidad_nulos)


id_artista 0
nombre_artista 0
pais 0
seguidores_spotify 0
seguidores_youtube 0
activo_desde 0


## 6) ELIMINAR DUPLICADOS

In [0]:
df_artistas.dropDuplicates()

df_musica.dropDuplicates()


DataFrame[id_cancion: string, nombre_cancion: string, id_artista: string, genero: string, anio_lanzamiento: int, popularidad: int]

## 7) VALIACIONES DE NEGOCIO

## TABLA SONG

## Análisis de Éxito Musical y Relevancia de Artistas,

In [0]:
from pyspark.sql.functions import col, when

# Creamos una nueva columna para ve el nivel de popularidad de la canción
df_musica = df_musica.withColumn(
    "nivel_popularidad",
    when((col("popularidad") >= 90) & (col("popularidad") <= 100), "Hit")
    .when((col("popularidad") >= 70) & (col("popularidad") <= 89), "Regular")
    .when((col("popularidad") >= 40) & (col("popularidad") <= 60), "Bajo")
    .otherwise("Muy bajo")
)

df_musica.display()

id_cancion,nombre_cancion,id_artista,genero,anio_lanzamiento,popularidad,nivel_popularidad
S001,Shake It Off,AR001,Pop,2014,95,Hit
S002,Shape of You,AR002,Pop,2017,98,Hit
S003,Bad Guy,AR003,Pop,2019,92,Hit
S004,Blinding Lights,AR004,R&B,2019,99,Hit
S005,Thank U Next,AR005,Pop,2018,91,Hit
S006,Titi Me Pregunto,AR006,Regueton,2022,96,Hit
S007,Gods Plan,AR007,Hip Hop,2018,94,Hit
S008,Lose Yourself,AR008,Hip Hop,2002,93,Hit
S009,Sorry,AR009,Pop,2015,90,Hit
S010,Umbrella,AR010,Pop,2007,89,Regular


### Ciclo de Vida de la Canción por fecha de lanzamiento

In [0]:
from pyspark.sql.functions import col, when
df_musica = df_musica.withColumn(
    "ciclo_vida_cancion",
    when(col("anio_lanzamiento") == 2026, "Estreno Reciente")
    .when((col("anio_lanzamiento") >= 2015) & (col("anio_lanzamiento") <= 2025), "Catalogo Moderno")
    .otherwise("Catalogo Clasico")
)

df_musica.display()

id_cancion,nombre_cancion,id_artista,genero,anio_lanzamiento,popularidad,nivel_popularidad,ciclo_vida_cancion
S001,Shake It Off,AR001,Pop,2014,95,Hit,Catalogo Clasico
S002,Shape of You,AR002,Pop,2017,98,Hit,Catalogo Moderno
S003,Bad Guy,AR003,Pop,2019,92,Hit,Catalogo Moderno
S004,Blinding Lights,AR004,R&B,2019,99,Hit,Catalogo Moderno
S005,Thank U Next,AR005,Pop,2018,91,Hit,Catalogo Moderno
S006,Titi Me Pregunto,AR006,Regueton,2022,96,Hit,Catalogo Moderno
S007,Gods Plan,AR007,Hip Hop,2018,94,Hit,Catalogo Moderno
S008,Lose Yourself,AR008,Hip Hop,2002,93,Hit,Catalogo Clasico
S009,Sorry,AR009,Pop,2015,90,Hit,Catalogo Moderno
S010,Umbrella,AR010,Pop,2007,89,Regular,Catalogo Clasico


## TABLA ARTISTAS

In [0]:
df_artistas.display()

id_artista,nombre_artista,pais,seguidores_spotify,seguidores_youtube,activo_desde
AR001,Taylor Swift,Estados Unidos,150000000,43000000,2006
AR002,Ed Sheeran,Reino Unido,124000000,37000000,2011
AR003,Billie Eilish,Estados Unidos,122000000,44000000,2015
AR004,The Weeknd,Canada,117000000,30000000,2010
AR005,Ariana Grande,Estados Unidos,109000000,46000000,2008
AR006,Bad Bunny,Puerto Rico,106000000,10000000,2015
AR007,Drake,Canada,106000000,27000000,2006
AR008,Eminem,Estados Unidos,106000000,36000000,1999
AR009,Justin Bieber,Canada,86500000,67000000,2009
AR010,Rihanna,UNKNOWN,70900000,23000000,2005


### Nivel de Influencia de Seguidores de Spotify y Yotube

In [0]:
df_artistas = df_artistas.withColumn(
    "seguidores_totales",
    col("seguidores_spotify") + col("seguidores_youtube")
)

In [0]:
df_artistas = df_artistas.withColumn(
    "nivel_influencia",
    when(col("Seguidores_Totales") >= 5000000, "Leyenda")
    .when((col("Seguidores_Totales") >= 1000000) & (col("Seguidores_Totales") <= 4999999), "Consolidado")
    .otherwise("Emergente")
)

df_artistas.display()

id_artista,nombre_artista,pais,seguidores_spotify,seguidores_youtube,activo_desde,seguidores_totales,nivel_influencia
AR001,Taylor Swift,Estados Unidos,150000000,43000000,2006,193000000,Leyenda
AR002,Ed Sheeran,Reino Unido,124000000,37000000,2011,161000000,Leyenda
AR003,Billie Eilish,Estados Unidos,122000000,44000000,2015,166000000,Leyenda
AR004,The Weeknd,Canada,117000000,30000000,2010,147000000,Leyenda
AR005,Ariana Grande,Estados Unidos,109000000,46000000,2008,155000000,Leyenda
AR006,Bad Bunny,Puerto Rico,106000000,10000000,2015,116000000,Leyenda
AR007,Drake,Canada,106000000,27000000,2006,133000000,Leyenda
AR008,Eminem,Estados Unidos,106000000,36000000,1999,142000000,Leyenda
AR009,Justin Bieber,Canada,86500000,67000000,2009,153500000,Leyenda
AR010,Rihanna,UNKNOWN,70900000,23000000,2005,93900000,Leyenda


### Dominio de Plataforma

In [0]:
df_artistas = df_artistas.withColumn(
    "dominio_plataforma",
    when(col("seguidores_youtube") > col("seguidores_spotify"), "Artista de Youtube")
    .when(col("seguidores_spotify") > col("seguidores_youtube"), "Artista de Spotify")
    .otherwise("Equilibrado")

)

df_artistas.display()

id_artista,nombre_artista,pais,seguidores_spotify,seguidores_youtube,activo_desde,seguidores_totales,nivel_influencia,dominio_plataforma
AR001,Taylor Swift,Estados Unidos,150000000,43000000,2006,193000000,Leyenda,Artista de Spotify
AR002,Ed Sheeran,Reino Unido,124000000,37000000,2011,161000000,Leyenda,Artista de Spotify
AR003,Billie Eilish,Estados Unidos,122000000,44000000,2015,166000000,Leyenda,Artista de Spotify
AR004,The Weeknd,Canada,117000000,30000000,2010,147000000,Leyenda,Artista de Spotify
AR005,Ariana Grande,Estados Unidos,109000000,46000000,2008,155000000,Leyenda,Artista de Spotify
AR006,Bad Bunny,Puerto Rico,106000000,10000000,2015,116000000,Leyenda,Artista de Spotify
AR007,Drake,Canada,106000000,27000000,2006,133000000,Leyenda,Artista de Spotify
AR008,Eminem,Estados Unidos,106000000,36000000,1999,142000000,Leyenda,Artista de Spotify
AR009,Justin Bieber,Canada,86500000,67000000,2009,153500000,Leyenda,Artista de Spotify
AR010,Rihanna,UNKNOWN,70900000,23000000,2005,93900000,Leyenda,Artista de Spotify


### VIGENCIA DEL ARTISTA

In [0]:
from pyspark.sql.functions import col, when, lit  # lit para crear constantes
from datetime import datetime  # Importamos la clase datetime

anio_actual = datetime.now().year # Obtenemos el año actual

df_artistas = df_artistas.withColumn(
    "vigencia_artista",
    when((lit(anio_actual) - col("activo_desde")) > 15, "Veterano")
    .when((lit(anio_actual) - col("activo_desde")) < 3, "Revelación")
    .otherwise("Contemporáneo")
)

df_artistas.display()

id_artista,nombre_artista,pais,seguidores_spotify,seguidores_youtube,activo_desde,seguidores_totales,nivel_influencia,dominio_plataforma,vigencia_artista
AR001,Taylor Swift,Estados Unidos,150000000,43000000,2006,193000000,Leyenda,Artista de Spotify,Veterano
AR002,Ed Sheeran,Reino Unido,124000000,37000000,2011,161000000,Leyenda,Artista de Spotify,Contemporáneo
AR003,Billie Eilish,Estados Unidos,122000000,44000000,2015,166000000,Leyenda,Artista de Spotify,Contemporáneo
AR004,The Weeknd,Canada,117000000,30000000,2010,147000000,Leyenda,Artista de Spotify,Veterano
AR005,Ariana Grande,Estados Unidos,109000000,46000000,2008,155000000,Leyenda,Artista de Spotify,Veterano
AR006,Bad Bunny,Puerto Rico,106000000,10000000,2015,116000000,Leyenda,Artista de Spotify,Contemporáneo
AR007,Drake,Canada,106000000,27000000,2006,133000000,Leyenda,Artista de Spotify,Veterano
AR008,Eminem,Estados Unidos,106000000,36000000,1999,142000000,Leyenda,Artista de Spotify,Veterano
AR009,Justin Bieber,Canada,86500000,67000000,2009,153500000,Leyenda,Artista de Spotify,Veterano
AR010,Rihanna,UNKNOWN,70900000,23000000,2005,93900000,Leyenda,Artista de Spotify,Veterano


# 8) Convertir la tabla Song de bronze a tabla Silver

In [0]:
# convertir la data songs a tabla 
df_musica.write.mode("overwrite") \
    .saveAsTable("proyecto_spotify.silver_songs")


In [0]:
# convertir la data artist a tabla 
df_artistas.write.mode("overwrite") \
    .saveAsTable("proyecto_spotify.silver_artists")
